# "Should you use synthetic data for label balancing"

After having worked with tabular synthetic data for the past 6 months, I have encountered many articles claiming that synthetic data is the solution for many (machine learning) problems. While synthetic data can serve as a useful Privacy Enhancing Technology (PET) and has shown to be useful in certain tasks, its usefulness and relevance is not always clearly assessed. An example of this is, and also the inspiration for me writing this article, is the article provided by Synthetic Data Vault (SDV) titled: "Can you use synthetic data for label balancing?" (https://sdv.dev/blog/synthetic-label-balancing/).

The article addresses a well-known issue in classification problems: imbalanced target labels. The article correctly identifies techniques like Random Oversampling (ROS) and noise injection while acknowledging their downsides (being overfitting and noisy data). However, the article then presents synthetic data as a "compelling solution" without providing enough evidence of this. While I am a fan of SDV, their generators, preprocessors, and constraints, this article overlooks important aspects validating the validity of synthetic data for these problems. Although you definitely can use synthetic data for label balancing (to answer the question of the article), the key question is whether you **should** use synthetic data and how it compares to state-of-the-art (SOTA) techniques.

Throughout this article, I aim to provide an answer to this question by comparing synthetic data produced by SDV generators against alternatives and build on top of the aformentioned article. Specifically, I compare data-level approaches such as noise injection, ROS, Synthetic Minority Over-sampling TEchnique (SMOTE), and CTGAN against the algorithm-level approach of Cost-Sensitive learning. This idea is not novel and adjacent research is available in literature. Adiputra and Wanchai (2024) for instance compare similar approaches resampling and synthetic data approaches. However, their calidation approach uses cross validation while resampling before cross validation, which is a common pitfall leading to data leakage. Also a mistake in section 5.7.4 of: https://d2l.ai/chapter_multilayer-perceptrons/kaggle-house-price.html. It is not the exact same mistake, but similar. Transformations should be applied separately so doesnt really

This article aims to improve on this by providing a more methodologically sound approach whilst providing the intuition and explanation for practictioners that are less familiar with imbalanced classification problems.

### Notes

- Check the helper_functions
    - move _get_num_samples to init?
    - Fix scoring function for results
    - Should I use fit/transform as opposed to fit_resample/resample in the custom samplers


## Imports

Notably, to ensure a proper cross validation procedure, I use a pipeline. Specifically, the pipeline from imb_learn is used over sklearn's

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt     
import seaborn as sns

import lightgbm as lgb

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.pipeline import Pipeline

from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split

from sdv.metadata import Metadata
from sdv.single_table import CTGANSynthesizer, TVAESynthesizer, GaussianCopulaSynthesizer

from helper_functions import NoiseSampler, ColumnScaler, SDVSampler, display_scores

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Data exploration

For this analysis, the creditcard dataset will be used from Kaggle (https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud), containing transactions and whether they were fraudulent or not. The goal is to predict whether a transaction is fraudulent or not, making it a classification task. Naturally, the amount of non-fraudulent transactions outweigh the number of fraudulent transactions resulting in an imbalanced classification task.

For this analysis, we will only be using a subset of the columns, which in this case contains mostly the Principal Components of the original data for confidentiality reasons. In addition, we also have a variable called "Time" which is the seconds from the first transaction in the dataset, 'Amount' which is the amount spent on the transaction, and 'Class' indicating whether the transactions is fraudulent or not, which also is our target variable. Ultimately, the data consists mostly of floats with the exception of our target which is binary.

In [3]:
# Select first then and last 2 columns
creditcard = pd.read_csv('../data/creditcard.csv')

# To reduce the dimensiolality of the dataset, we will only use the first 12 principal components,
# Time, Amount, and target
print(creditcard.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

Next, we see how our target labels are distributed.

In [4]:
# Target label distribution rounded to 2 decimal places
round(creditcard['Class'].value_counts(normalize = True) * 100, 2)

Class
0    99.83
1     0.17
Name: proportion, dtype: float64

It becomes evident that the dataset is highly imbalanced, with  only 0.17% of transactions being fraudulent. This can be problematic for machine learning algorithms if you were to not account for this as models tend to bias towards the non-fraudulent cases given their overrepresentation in the data. As a result, a model predicting every transaction to be non-fraudulent would already result in a 99% accuracy even though every fraudulent transaction has been wrongfully predicted. This highlights the importance of carefully selecting evaluation metrics, as accuracy alone can be misleading in imbalanced classification problems.

### Data splitting

Next step is to split the data. We stratify on the target variable to ensure an even split across train/set sets.

In [5]:
X = creditcard.drop('Class', axis = 1)
y = creditcard['Class']

## Metrics

As mentioned previously, the choice of metrics is far from trivial. However, given that it is beyond the scope of this blog, I will choose an F1-score without diving too deep into the costs of misclassification.

### Algorithm

For the algorithm, I will use a LGBM Classifier. Choosing the most optimal estimator is beyond the scope of this blog. LGBM is chosen for its efficiency and relative predictive power, therefore being used consistently.

## Settings

We define the folds and parameters to optimize over as these will be consistent across resampling techniques.

In [6]:
RANDOM_STATE = 2

CV_FOLDS = StratifiedKFold(n_splits = 5, random_state = RANDOM_STATE, shuffle = True)

SCORINGS = ['f1', 'roc_auc', 'precision', 'recall']

## Incorrect cross validation procedure

Suppose we wish to use ROS to oversample the fraud instances. A common mistake that is made is to perform this oversampling before splitting the data. Why is this wrong: "Animation here". To illustrate this we will test two predictive models on truly unseen data. Therefore, for this section we will make use of a train test split. The train set is used to determine the cross validation scores. Next, the cross validation scores are compared. Next predictions are made on the test set and then the predictions on the test set are compared.



The cross validation mistake is easily made and looks like this implemented in python:

In [7]:
# Split the dataset into training and testing
X_cv, X_test, y_cv, y_test = train_test_split(X, y, test_size = 0.2, stratify = y,
                                                    random_state = RANDOM_STATE)

In [8]:
# We resample the data used for cross-validation
ros = RandomOverSampler(random_state = RANDOM_STATE)
X_res, y_res = ros.fit_resample(X_cv, y_cv)

cv_score = cross_validate(
    lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1),
    X_res, y_res, cv = CV_FOLDS, scoring = SCORINGS, n_jobs = -1
)

cv_score = cv_score['test_f1'].mean()
print(f"Cross validation score: {cv_score:.4f}")

Cross validation score: 0.9999


This approach leads to (near) perfect scores. Hopefully, this raises some suspicison as to the validity of the results. This seems great, but how do these results translate to truly unseen data: the test set we have not used in the RandomOverSampler.


In [9]:
# We define the model and fit it to the resampled cv data
lgbm_classifier = lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, 
                                     verbose = -1)
lgbm_classifier.fit(X_res, y_res)

# predictions on the unseen test data and evaluation
preds = lgbm_classifier.predict(X_test)
test_score = f1_score(y_test, preds)
print(f"Test score: {test_score:.4f}")


Test score: 0.8442


We can see that the test results are lower than the cross validation results indicating that the cross validation score in this way is not representative. Typically you also want to tune hyperparameters during cross validation using GridSearch for example. In this case, this might lead to even worse results because hyperparameters are chosen during incorrect cross validation procedure and fit to that which might results in worse generalization.

Although apparent in this situation given the near perfect CV scores, mostly as a results of the severe class imbalance (many observations are duplicates), this won't always be the case with every dataset. In addition, there are other ways of data leakage that are more subtle (explain feature engineering ( including male/female --> 1/0)). Best practice to do everything for train/test separately

#### Random OverSampling the Correct Way

Next up, we perform cross validation the correct way. There are multiple ways to do it. Namely, iterating over each fold in a loop and manually applying the required transformations to the train and holdout set within each fold separately or using a pipeline. For simplicity sake, we will be using the latter option.

In [10]:
# Define the sampling pipeline
pipeline_ros = Pipeline(
    [('ros', RandomOverSampler(random_state = RANDOM_STATE)),
     ('LGB', lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1))]
)

# Now we apply the cross-validation on the pipeline using the non resampled data
# Subsequently the pipeline will resample the data for each fold separately
cv_score_ros = cross_validate(pipeline_ros, X_cv, y_cv, cv = CV_FOLDS, scoring = SCORINGS, 
                               n_jobs = -1)

cv_score_pipe = cv_score_ros['test_f1'].mean()
print(f"Cross validation score with pipeline: {cv_score_pipe:.4f}")

Cross validation score with pipeline: 0.8384


Although the score is less impressive than the first method, it more accurately reflects the results on the holdout set of 0.8442. Furthermore, (I forgot what I wanted to say)

## Why does this happen? What exactly is going wrong?

Compare and display duplicates. I can display duplicates through manual cross validation procedure (before and after)

The idea of creating a train and test split is to evaluate the model on data that it has not seen before. The same thing happens with cross validation. The hold out fold in each fold is used as a separate fold to test the data on. Since the hold out fold differs at each iteration

In [11]:
# Isolate the positive class of the original unresampled dataset
X_pos = X[y == 1]
prop_dupl = X_pos.duplicated(keep = False).sum() / X_pos.shape[0]
print(f"Duplicates in positive class: {prop_dupl:.4f}")

Duplicates in positive class: 0.0650


This indicates that approx. 6.5% of observations are not unique.

Using stratification across the folds, we would expect a similar percentage throughout cross validation. First, the number of duplicates are presented across folds with the data that is resampled beforehand, i.e. the X_res and y_res from earlier. Now, the cross validation process is performed manually, and for each iteration we look into how many duplicates there are between train and holdout within each fold.

In [ ]:
# We track the proportion of duplicates in the test set across folds and average them
dupl_percentages = []

for i, (train_index, hd_index) in enumerate(CV_FOLDS.split(X_res, y_res)):
    
    # Obtain train and holdout sets for the current fold
    X_train, X_hd = X_res.iloc[train_index], X_res.iloc[hd_index]
    y_train, y_hd = y_res.iloc[train_index], y_res.iloc[hd_index]
    
    train_pos = X_train[y_train == 1]
    hd_pos = X_hd[y_hd == 1]
    
    # Check to see the observations in holdout that are duplicates of the training set
    mask = hd_pos.apply(tuple, axis = 1).isin(train_pos.apply(tuple, axis = 1))
    duplicates = hd_pos[mask]
    
    # determine the proportion of holdout observation that are duplicates
    prop_dupl = duplicates.shape[0] / hd_pos.shape[0] * 100
    dupl_percentages.append(prop_dupl)
    
print(f"Proportion of duplicates in holdout set across folds: {np.mean(dupl_percentages):.2f}%")


Proportion of duplicates in holdout set across folds: 100.00%


We see that the average duplicates across folds is 100% if we resample before performing cross validation, meaning that all of the fraud instances in the holdout set are duplicates of observations in the train set the model would be trained on, leading to the near perfect test F1-score we saw earlier on. These are also likely more extreme scores given the severe class imbalance in this dataset. This approach is not representative because observations in the test set should be unseen by the model and should also not be synthetic or resampled as that would not be how a real model would be used/tested. Therefore, the holdout set should never contain resampled observations. How you apply this manually is displayed below.

We now repeat the same steps as above, i.e. checking the number of duplicates in the test set, but now we use the proper way of cross validation and applying resampling WITHIN each fold as opposed to beforehand.

In [13]:
# Again, we track the proportion of duplicates in the holdout set across folds

dupl_percentages = []

for i, (train_index, hd_index) in enumerate(CV_FOLDS.split(X_cv, y_cv)):
   
    # Obtain train and holdout sets for the current fold
    X_train, X_hd = X_cv.iloc[train_index], X_cv.iloc[hd_index]
    y_train, y_hd = y_cv.iloc[train_index], y_cv.iloc[hd_index]
    
    # Within each fold, we resample the training set
    X_res, y_res = ros.fit_resample(X_train, y_train)
    
    # Only then do we isolate the positive class
    train_pos = X_res[y_res == 1]
    hd_pos = X_hd[y_hd == 1]
    
    # Check to see the observations in holdout that are duplicates of the training set
    mask = hd_pos.apply(tuple, axis = 1).isin(train_pos.apply(tuple, axis = 1))
    duplicates = hd_pos[mask]
    
    # determine the proportion of holdout observation that are duplicates
    prop_dupl = duplicates.shape[0] / hd_pos.shape[0] * 100
    dupl_percentages.append(prop_dupl)
    
print(f"Proportion of duplicates in test set across folds: {np.mean(dupl_percentages):.2f}%")

Proportion of duplicates in test set across folds: 6.35%


The effect of improper cross validation procedure in this example with ROS is clear due to the high amount of data leakage and the exact duplicates. However, when using synthetic data generators such as the ones used in SDV, this might be less obvious depending on how much they overfit to the data. This also varies depending on the generator. As some generator tend to produce more direct duplicates of the original training data than others. 

However, this issue of data leakge between train and test is not something that only pertains to resampling methods in imbalanced classification tasks. This is also valid for other feature engineering methods that use information from observations that will end up in the hold out fold during cross validation. Think of scaling/normalizing, mean imputation etc. which would also lead to data leakage. 

Though not all feature engineering has to lead to data leakage between train and fold. Think of encoding a categorical gender variable from male/female to binary, which typically is fine since the information is contained within the rows. However, this might get a bit tricky when working with a high cardinality categorical variable where certain values are very rare meaning that they might occur in the hold out fold but not in the train fold. Similarly with word embeddings based on words that end up in the holdout fold but not in the train fold. For this reason, you should always be aware of the transformations you are doing, when you are doing them, and which information is used in those transformations. Therefore, it is best practice to perform all data transformations within the cross validation procedure as opposed to beforehand.

# Performances on imbalanced classification tasks

As mentioned previously, we compare the performances based on 

## Baseline performance

As a baseline for this problem to compare predictions to, we will use the DummyClassifier from sklearn. We use 'stratified' becasue choosing 'most_frequent' is probably also a natural choice, but given that we mostly evaluate on F1, all scores would be zero, making the improvement over the baseline seem better more impressive.

#### Perhaps move this section to later? after the explanation of what goes wrong for instance?

In [14]:
pipeline_baseline = Pipeline(
    [('dummy', DummyClassifier(strategy = 'stratified', 
                               random_state = RANDOM_STATE))]
)

cv_score_base = cross_validate(pipeline_baseline, X, y, cv = CV_FOLDS, scoring = SCORINGS, 
                                n_jobs = -1)

## Random OverSampling

Now on the entire dataset during cross validation instead of a split

In [15]:
# Define the sampling pipeline
pipeline_ros = Pipeline(
    [('ros', RandomOverSampler(random_state = RANDOM_STATE)),
     ('LGB', lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1))]
)

cv_score_ros = cross_validate(pipeline_ros, X, y, cv = CV_FOLDS, scoring = SCORINGS, 
                               n_jobs = -1)

## Noise injection

The first approach mentioned in the article is noise injection. Although I have not seen it being used in practice and the article does not mention the noise generating process, a uniform sampling procedure will be used. Specifically, for each variable I will extract their minimum and maximum values and use them to sample from a uniform distribution. Therefore, the correlations between variables are overlooked and the bivariate distributions won't be correct.

Given that all variables, with exception of the target variable, are numerical this step is quite straightforward.

In [16]:
# Define the sampling pipeline
pipeline_noise = Pipeline(
    [('noise_sampler', NoiseSampler(random_state = RANDOM_STATE)),
     ('LGB', lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1))]
)

cv_score_noise = cross_validate(pipeline_noise, X, y, cv = CV_FOLDS, scoring = SCORINGS, 
                                n_jobs = -1)

## SMOTE

In [17]:
# Define the sampling pipeline
pipeline_smote = Pipeline(
    [('scaler', ColumnScaler(['Amount', 'Time'])),
     ('smote', SMOTE(random_state = RANDOM_STATE)),
     ('LGB', lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1))]
)

cv_score_smote = cross_validate(pipeline_smote, X, y, cv = CV_FOLDS, scoring = SCORINGS, 
                                n_jobs = -1)


## SDV Generators

The SDV generators that will be compared are the CTGAN and TVAE. Also use Gaussian Copula?? depending on the distributions of the data

In [18]:
metadata = Metadata.detect_from_dataframe(X)

### Gaussian Copula

Finally, I will use the Gaussian Copula. The Gaussian Copula is a probabilistic model, making it more easily understood than the more complex GANs and VAEs. 

After a short inspection of the distributions, the gamma distribution is used as a default distribution for all variables in favour of choosing separate best fitting disitrbutions of for each variable specifically and using the kernal density estimation to save time.

In [ ]:
# Define the sampling pipeline
pipeline_gc = Pipeline(
    [('gc', SDVSampler(GaussianCopulaSynthesizer, metadata, random_state = RANDOM_STATE,
                       def_distr = 'gamma')),
     ('LGB', lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1))]
)

cv_score_gc = cross_validate(pipeline_gc, X, y, cv=CV_FOLDS, scoring = SCORINGS,
                             n_jobs = -1)

### CTGAN

I will use the CTGAN synthesizer in this case with default parameters and without any constraints.

In [19]:
# Define the sampling pipeline
pipeline_ctgan = Pipeline(
    [('ctgan', SDVSampler(CTGANSynthesizer, metadata, random_state = RANDOM_STATE)),
     ('LGB', lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1))]
)

cv_score_ctgan = cross_validate(pipeline_ctgan, X, y, cv = CV_FOLDS, scoring = SCORINGS, 
                               n_jobs = -1)

### TVAE Synthesizer

I will also use the TVAE Synthesizer. Again, there will be no hyperparameter tuning and no constraints.

In [20]:
# Define the sampling pipeline
pipeline_tvae = Pipeline(
    [('tvae', SDVSampler(TVAESynthesizer, metadata, random_state = RANDOM_STATE)),
     ('LGB', lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1))]
)

cv_score_tvae = cross_validate(pipeline_tvae, X, y, cv=CV_FOLDS, scoring = SCORINGS,
                               n_jobs = -1)

## Cost sensitive learning

I will be using the inverse class frequency for this as the real costs associated with misclassifying creditcard fraud is not known in this example. These weights will be specified using the classifier's 'class_weight' parameter. Specifically, this is set to 'balanced' to achieve the inversely proportional weights. 

Given that the folds are stratified, we assume the that the assigned weights for the entire train set is (roughly) equal to the weights for each fold. This will result in the following weights being assigned:


In [22]:
X.shape[0] / (2 * np.bincount(y)) 

array([  0.50086524, 289.43800813])

The pipeline then becomes

In [23]:
# Define the sampling pipeline
pipeline_csl = Pipeline(
    [('LGB', lgb.LGBMClassifier(class_weight= 'balanced', random_state = RANDOM_STATE,
                                n_jobs = -1, verbose = -1))]
)

cv_score_csl = cross_validate(pipeline_csl, X, y, cv=CV_FOLDS, scoring = SCORINGS,
                               n_jobs = -1)

# Results

In [24]:
cv_scores = [cv_score_base, cv_score_noise, cv_score_ros, cv_score_smote,
             cv_score_ctgan, cv_score_tvae, cv_score_gc, cv_score_csl]   

In [25]:
# Complete the scores of 
r = display_scores(cv_scores, SCORINGS)
r

,F1,Roc_auc,Precision,Recall
Baseline,0.000,0.499,0.000,0.000
Noise,0.708,0.974,0.626,0.819
ROS,0.837,0.975,0.862,0.815
SMOTE,0.654,0.961,0.538,0.837
CTGAN,0.764,0.971,0.721,0.817
TVAE,0.763,0.967,0.715,0.821
Gaussian Copula,0.847,0.978,0.906,0.797
CSL,0.839,0.975,0.843,0.835


Gaussian copula provides the best F1, which is kind of surprising. This is likely very dataset dependent and to rule out/for synthetic data, this process should in actuality be repeated for multiple datasets, which I don't have the time to do. Perhaps I should add a quick section: 'Understanding the data', since the dataset at hand might be over simplistic.

However, the Gaussian Copula provides the worsy recall of all methods (and best precision). For the other methods, this is more aligned with each other. So, it would also still be dependent on how you want to use the model and the cost of misclassification.